In [1]:
# Importando as bibliotecas
import bs4 
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas as pd
import re


In [2]:

def extrair_anuncios(url_base, pages=240,nome_arquivo='apartamentos.json'):
    # Declarando variável cards
    cards = []

    # Iterando por todas as páginas do site
    for i in range(1, pages + 1):
        # Obtendo o HTML
        url = f'{url_base}&pagina={i}'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}
        
        requisicao = Request(url, headers=headers)

        try:
            response = urlopen(requisicao)
            html = response.read().decode('utf-8')
            soup = BeautifulSoup(html, 'html.parser')
        except HTTPError as e:
            print(f'HTTPError: {e.code} - {e.reason} ao processar a página {i}')
            continue
        except URLError as e:
            print(f'URLError: {e.reason} ao processar a página {i}')
            continue

        # Obtendo as Tags de interesse
        try:
            container = soup.find('section', {'class': 'l-container Result_result_section__XZEpJ'})
            if not container:
                print(f'Erro ao processar a página {i}: container não encontrado')
                continue
            anuncios = container.findAll('div', class_='l-card__wrapper')
        except AttributeError:
            print(f'Erro ao processar a página {i}: estrutura da página mudou')
            continue

        if not anuncios:
            print(f'Erro ao processar a página {i}: nenhum anúncio encontrado')
            continue

        # Coletando as informações dos CARDS
        for anuncio in anuncios:
            card = {}

            # Valor
            valor_p = anuncio.find('p', {'class': re.compile(r'l-text l-u-color-\S+ l-text--variant-heading-small l-text--weight-bold undefined')})
            if valor_p:
                valor_text = valor_p.get_text(strip=True).split('/')[0].strip()
                valor = valor_text.replace('R$', '').replace('.', '').replace(',', '')
                try:
                    card['valor'] = int(valor)
                except ValueError:
                    card['valor'] = 0
            else:
                card['valor'] = 0
            
            # Descrição
            descricao = anuncio.find('p', {'class': 'card__description'})
            card['descricao'] = descricao.get_text().replace('-', ' ').replace('\n', '') if descricao else 'Vazio'

            # Bairro
            bairro_tag = anuncio.find('h2', {'class': 'l-text l-u-color-neutral-28 l-text--variant-heading-small l-text--weight-medium card__address'})
            if bairro_tag:
                bairro = bairro_tag.get_text(strip=True)
                card['bairro'] = bairro
            else:
                card['bairro'] = 'Não disponível'

            # Endereço
            endereco_tag = anuncio.find('p', {'class': 'l-text l-u-color-neutral-28 l-text--variant-body-small l-text--weight-regular card__street'})
            if endereco_tag:
                endereco = endereco_tag.get_text(strip=True)
                card['endereco'] = endereco
            else:
                card['endereco'] = 'Não disponível'
            
            # Quartos
            quartos = anuncio.find('p', {'itemprop': 'numberOfRooms'})
            card['quartos'] = (quartos.get_text().replace(' ', '').replace('\n', '')) if quartos else 1

            # Garagem
            garagem = anuncio.find('p', {'itemprop': 'numberOfParkingSpaces'})
            card['garagem'] = (garagem.get_text().replace(' ', '').replace('\n', '')) if garagem else 0

            # Banheiros
            banheiro = anuncio.find('p', {'itemprop': 'numberOfBathroomsTotal'})
            card['banheiro'] = (banheiro.get_text().replace(' ', '').replace('\n', '')) if banheiro else 1

            # Metragem
            metro = anuncio.find('p', {'itemprop': 'floorSize'})
            card['area(m2)'] = (metro.get_text().replace(' ', '').replace('\n', '').replace('m²', '')) if metro else 30

            # IPTU e Condomínio
            iptu_condominio_tag = anuncio.find('p', {'class': 'l-text l-u-color-neutral-44 l-text--variant-body-small l-text--weight-regular undefined'})
            if iptu_condominio_tag:
                iptu_condominio_text = iptu_condominio_tag.get_text(strip=True)
                
                # Extraindo o valor do condomínio
                condominio_match = re.search(r'Cond\. R\$ (\d[\d.,]*)', iptu_condominio_text)
                if condominio_match:
                    condominio_text = condominio_match.group(1).replace('.', '').replace(',', '')
                    try:
                        card['condominio'] = int(condominio_text)
                    except ValueError:
                        card['condominio'] = 0
                else:
                    card['condominio'] = 0
                
                # Extraindo o valor do IPTU
                iptu_match = re.search(r'IPTU R\$ (\d[\d.,]*)', iptu_condominio_text)
                if iptu_match:
                    iptu_text = iptu_match.group(1).replace('.', '').replace(',', '')
                    try:
                        card['iptu'] = int(iptu_text)
                    except ValueError:
                        card['iptu'] = 0
                else:
                    card['iptu'] = 0
            else:
                card['iptu'] = 0
                card['condominio'] = 0
            
            # Suíte
            card['suite'] = 1 if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) else 0

            cards.append(card)

    # Criando o DataFrame
    dataset = pd.DataFrame(cards)
    # Exportando o DataFrame para um arquivo JSON
    dataset.to_json( nome_arquivo, orient='records', indent=4, force_ascii=False)
    print("DataFrame exportado para JSON com sucesso!")

In [ ]:
url_ap = 'https://www.zapimoveis.com.br/aluguel/apartamentos/mg+belo-horizonte/?__ab=exp-aa-test:B,super-high:new,olx:control,phone-page:control,sup-hl-pl:newC,off-no-hl:control&transacao=aluguel&onde=,Minas%20Gerais,Belo%20Horizonte,,,,,city,BR%3EMinas%20Gerais%3ENULL%3EBelo%20Horizonte,-20.024071,-44.059007,&tipos=apartamento_residencial'
extrair_anuncios(url_ap,pages=240,nome_arquivo='zap_ap.json')

In [109]:
url_cobertura = 'https://www.zapimoveis.com.br/aluguel/cobertura/mg+belo-horizonte/?__ab=exp-aa-test:B,super-high:new,olx:control,phone-page:control,sup-hl-pl:newC,off-no-hl:control&transacao=aluguel&onde=,Minas%20Gerais,Belo%20Horizonte,,,,,city,BR%3EMinas%20Gerais%3ENULL%3EBelo%20Horizonte,-20.024071,-44.059007,&tipos=cobertura_residencial&pagina=1'
extrair_anuncios(url_cobertura,pages=35,nome_arquivo='zap_cob.json')

DataFrame exportado para JSON com sucesso!


In [ ]:
#url_casa = 'https://www.zapimoveis.com.br/aluguel/casas/mg+belo-horizonte/?__ab=exp-aa-test:B,super-high:new,olx:control,phone-page:control,sup-hl-pl:newC,off-no-hl:control&transacao=aluguel&onde=,Minas%20Gerais,Belo%20Horizonte,,,,,city,BR%3EMinas%20Gerais%3ENULL%3EBelo%20Horizonte,-20.024071,-44.059007,&tipos=casa_residencial&pagina=1'
#extrair_anuncios(url_casa,pages=104,nome_arquivo='casas.json')